## 1.1. Run Radiography

In [ ]:
import Radiography_Library as RadLib; import platform

if platform.system() == "Darwin": directory = 'BUILD/'
if platform.system() == "Windows": directory = 'build\\Release\\'

threads = 10
energy = 80 # keV
sim_time = 60 # min
iteration_time = 30 # min

RadLib.RunRadiography(directory, threads, energy, sim_time, iteration_time)
if platform.system() == "Darwin": RadLib.PlayAlarm()

## 1.2. Run DEXA

In [ ]:
import Radiography_Library as RadLib; import platform

if platform.system() == "Darwin": directory = 'BUILD/'
if platform.system() == "Windows": directory = 'build\\Release\\'

threads = 10
sim_time = 10*60 # min
iteration_time = 30 # min

RadLib.RunDEXA(directory, threads, sim_time, iteration_time)
# if platform.system() == "Darwin": RadLib.PlayAlarm()

## 1.3. Merge Root Files

In [ ]:
import Radiography_Library as RadLib; import platform

directory = '/Users/miguelcomett/geant4-v11.2.2_2/ESTANCIA' + '/'
directory = 'BUILD/ROOT/'

starts_with = 'Rad_80'
output_name = 'Rad_80'

# trim_coords = (-250, 250, -270, 230)  # x_min, x_max, y_min, y_max
trim_coords = None

RadLib.MergeRoots_Parallel(directory, starts_with, output_name, trim_coords) 
# if platform.system() == "Darwin": RadLib.PlayAlarm()

## 1.4. Get ROOT Summary Data & Histograms 

In [ ]:
import Radiography_Library as RadLib

directory = 'BUILD/ROOT/'
directory = '/Users/miguelcomett/GEANT4/G4_MedX/'

root_file = 'Rad_40' + '.root'
root_file = 'thorax_40_2285.root'

data_tree = 'Hits'; data_branch_x = 'x_ax'; data_branch_y = 'y_ax'
summary_tree = 'Run Summary'; sumarry_branches = ['Number_of_Photons', 'EDep_Value_TeV', 'Radiation_Dose_uSv']

NumberofHits, NumberofPhotons, EnergyDeposition, RadiationDose = RadLib.Summary_Data(directory, root_file, data_tree, data_branch_x, summary_tree, sumarry_branches)

range_x = [-250, 250, 100] # range_min, range_max, bins (mm) 
range_y = [-280, 280, 100] # range_min, range_max, bins (mm)

RadLib.XY_1D_Histogram(directory, root_file, data_tree, data_branch_x, data_branch_y, range_x, range_y)

## 1.5. CT Summary

In [ ]:
import Radiography_Library as RadLib

directory = '/Users/miguelcomett/geant4-v11.2.2_2/ESTANCIA/CT/'

tree = 'Run Summary'
branches = ['Number_of_Photons', 'EDep_Value_TeV', 'Radiation_Dose_uSv']

NumberofPhotons, EnergyDeposition, RadiationDose = RadLib.CT_Summary_Data(directory, tree, branches)

## 2. Root File to Heatmap

In [ ]:
import Radiography_Library as RadLib

directory = 'BUILD/ROOT/'
directory = '/Users/miguelcomett/GEANT4/G4_MedX/'

root_name = 'thorax_40_2285' + ".root"
root_name = 'Rad_80' + '.root'

tree_name = "Hits"; x_branch  = "x_ax"; y_branch  = "y_ax"
size = [270, 270, 0, 10] # size_x, size_y, x_shift, y_shift, 
pixel_size = 0.5 # mm

heatmap_raw, xlim, ylim = RadLib.Root_to_Heatmap(directory, root_name, tree_name, x_branch, y_branch, size, pixel_size)
heatmap, firstrow = RadLib.Logaritmic_Transform(heatmap_raw)
RadLib.Plot_Heatmap(heatmap, '')

#### Show Interactive Plot

In [ ]:
import plotly.graph_objects as go; import plotly.io as pio

fig = go.Figure(go.Heatmap(z = heatmap, x = xlim, y = ylim, colorscale = [[0, 'black'], [1, 'white']], showscale = False))
fig.update_layout(width = 800, height = 800, yaxis = dict(autorange = 'reversed'))    
fig.show()

## 3. DEXA: Tissue Segmentation

### Calculate Heatmaps

In [12]:
import Radiography_Library as RadLib

directory = 'BUILD/ROOT/'
directory = '/Users/miguelcomett/GEANT4/G4_MedX/'

rootnames = ["thorax_40_2285.root", "thorax_80_2285.root"]
rootnames = ["Rad_40.root", "Rad_80.root"]

tree_name = "Hits"; x_branch  = "x_ax"; y_branch  = "y_ax"

size = [230, 230, 0, -20]

pixel_size = 0.5

low_energy_img, xlim, ylim  = RadLib.Root_to_Heatmap(directory, rootnames[0], tree_name, x_branch, y_branch, size, pixel_size)
high_energy_img, xlim, ylim = RadLib.Root_to_Heatmap(directory, rootnames[1], tree_name, x_branch, y_branch, size, pixel_size)
low_energy_img = RadLib.Logaritmic_Transform(low_energy_img)
high_energy_img  = RadLib.Logaritmic_Transform(high_energy_img)

### Save heatmaps as CSV

In [2]:
import Radiography_Library as RadLib

read_folder = '/Users/miguelcomett/GEANT4/G4_MedX/'

RadLib.Save_Heatmap_to_CSV(low_energy_img, read_folder, '40kev_Projection')
RadLib.Save_Heatmap_to_CSV(high_energy_img, read_folder, '80kev_Projection')

### Read heatmap's CSV

In [6]:
import Radiography_Library as RadLib

read_folder = '/Users/miguelcomett/GEANT4/G4_MedX/'

low_energy_img = RadLib.Read_Heatmap_from_CSV(read_folder, '40kev_Projection')
high_energy_img = RadLib.Read_Heatmap_from_CSV(read_folder, '80kev_Projection')

### Perform Segmentation

In [ ]:
import Radiography_Library as RadLib

sigma = 0
wn = 1

save_in = ''
save_as = ['', '', '', '', '', '', '', ''] # Low Energy, High Energy, Bone [SLS], Tissue [SLS], Bone [SSH], Tissue [SSH], Bone [ACNR], Tissue [ACNR]
save_all = ''

SLS_Bone, SLS_Tissue, SSH_Bone, SSH_Tissue, ACNR_Bone, ACNR_Tissue = RadLib.IsolateTissues(low_energy_img, high_energy_img, sigma, sigma, wn, save_in, save_as, save_all)

## 4. Bone Mineral Density (BMD)

In [ ]:
import Radiography_Library as RadLib

save_as = ''
thickness_bone = RadLib.BMO(SLS_Bone, SLS_Tissue, save_as)

#### Plot Interactive Heatmap

In [ ]:
import plotly.graph_objects as go; import plotly.io as pio

fig = go.Figure(go.Heatmap(z = thickness_bone, x = xlim, y = ylim))
fig.update_layout(width = 400, height = 400, xaxis = dict(autorange = 'reversed'), yaxis = dict(autorange = 'reversed'))    
fig.show()

## 5.1. Calculate Interactive CNR

#### Trim Image

In [ ]:
from PIL import Image; import matplotlib.pyplot as plt

directory = 'RESULTS/'
image = 'ssh' + '.png'
image = Image.open(directory + image)
image = image.convert('L')

print(image.size)
width = image.width; height = image.height

trim = 200
# image = image.crop((trim, trim, width - trim, height - trim)) # left, top, right, bottom
# image = image.crop((8410, trim, width - 60, height - trim))

# plt.imshow(image, cmap='gray'); plt.axis('off'); plt.show()

#### Launch Interactive CNR

In [ ]:
%matplotlib widget 
%matplotlib tk
import Radiography_Library as RadLib

RadLib.Interactive_CNR(image)

## 5.2. Calculate Fixed CNR 

In [ ]:
import Radiography_Library as RadLib

image_path = "RESULTS/" + "a" + ".png"
save_as = ''

shftx_s = 0.0 # shift x-coordinate signal box
shfty_s = 0.0 
shftx_b = 200.0 # shift x-coordinate background box
shfty_b = 0.0

coords_signal  = [1200 + shftx_s, 1000 + shfty_s, 1800 + shftx_s, 1800 + shfty_s] # x1, y1, x2, y2
coords_bckgrnd = [2100 + shftx_b, 1000 + shfty_b, 2300 + shftx_b, 1800 + shfty_b] # x1, y1, x2, y2

RadLib.Fixed_CNR(image_path, save_as, coords_signal, coords_bckgrnd)

## 6.1. Denoise with Skimage.Denoise_Bilateral

In [ ]:
import Radiography_Library as RadLib

# load array
path = SSH_Bone
isArray = True

# or load image
if isArray == False:
    directory = 'RESULTS/'
    path = directory + 'a' + '.png'

sigma_color = 0.05
sigma_spatial = 20

Denoised_Image = RadLib.Denoise_EdgeDetection(path, isArray)

## 6.2. Denoise by Fourier Transform

In [ ]:
import Radiography_Library as RadLib

array = heatmap
isHann = False

alpha = 1

save_as = ''
isCrossSection = False # plot crosss-section

fft_image = RadLib.Denoise(array, isHann, alpha, save_as, isCrossSection)

## 7. Save Plotly Heatmap

In [ ]:
import Radiography_Library as RadLib

# array = htmp_array
# array = Denoised_Image
array = ACNR_Bone
array = low_energy_img

title   = r"$ \large{ \text{Thorax Radiography Projection(40 keV)} } $"
x_label = r"$ \large{ \text{X Axis} \ (mm)} $"
y_label = r"$ \large{ \text{Y Axis} \ (mm)} $"

width  = 800
height = 800

save_as = ''

RadLib.Plotly_Heatmap_1(array, xlim, ylim, title, x_label, y_label, width, height, save_as)

### Plot with annotation and rectanngles

In [ ]:
import Radiography_Library as RadLib

array = heatmap

title   = r"$ \large{ \text{Low energy projection (40 keV), 100M Photons} } $"
x_label = r"$ \large{ \text{X Axis} \ (mm)} $"
y_label = r"$ \large{ \text{Y Axis} \ (mm)} $"

sqr_1_coords = [10, 10, -10, -10]
sqr_2_coords = [10, 10, -10, -10]

annotation = 'CNR = ' 

width  = 700
height = 700

save_as = ''

Radlib.Plotly_Heatmap_2(array, xlim, ylim, title, x_label, y_label, sqr_1_coords, sqr_2_coords, annotation, width, height, save_as)

# 8. CT Scan

## 8.1 Run CT loop

In [ ]:
import Radiography_Library as RadLib; import platform

if platform.system() == "Darwin": directory = 'BUILD/'
if platform.system() == "Windows": directory = 'build\\Release\\'

starts_with = 'CT'

angles = [277, 360]

RadLib.CT_Loop(directory, starts_with, angles)

## 8.2 Calculate projections at every angle from roots and save to CSV

In [ ]:
import Radiography_Library as RadLib; import matplotlib.pyplot as plt; import numpy as np

directory = '/Users/miguelcomett/geant4-v11.2.2_2/G4_MedX/CT_Raw' + '/'
filename = 'Aang'

roots = [0, 359, 1] # start, end, degree step

tree_name = "Hits"
x_branch  = "x_ax"
y_branch  = "y_ax"

dimensions = [270, 270, 0, 0] # sizeX, sizeY, shiftX, shiftY, # in mm

pixel_size = 0.5 # in mm

csv_folder = '/Users/miguelcomett/geant4-v11.2.2_2/G4_MedX/CT_CSV'

htmp, xlim, ylim = RadLib.Calculate_Projections(directory, filename, roots, tree_name, x_branch, y_branch, dimensions, pixel_size, csv_folder)

print("Max value:", np.max(htmp))
print("99th percentile:", np.percentile(htmp, 99))

lower = np.percentile(htmp, 0)
upper = np.percentile(htmp, 99)
clipped_htmp = np.clip(htmp, lower, upper)
plt.imshow(clipped_htmp, cmap='gray'); plt.colorbar(); plt.show()

# RadLib.Plot_Heatmap(htmp, xlim, ylim, save_as='')

## 8.3 Load projections from CSV and perfrom CT reconstruction

In [ ]:
import Radiography_Library as RadLib; import numpy as np; from scipy import ndimage; from tqdm import tqdm

roots = [0, 359, 1] # start, end, degree step
csv_folder = '/Users/miguelcomett/geant4-v11.2.2_2/G4_MedX/CT_CSV' + '/'
radiographs = RadLib.LoadHeatmapsFromCSV(csv_folder, roots)

In [ ]:
dimensions = [260, 260, 0, 0] # sizeX, sizeY, shiftX, shiftY, # in mm
pixel_size = 0.7
sigma = 3
htmps = np.zeros(len(radiographs), dtype = 'object')
for i, radiograph in tqdm(enumerate(radiographs), desc = 'Computing logarithmic transformation', unit = ' Heatmaps', leave = True):
    radiograph = ndimage.gaussian_filter(radiograph, sigma)
    htmp_array, xlim, ylim = RadLib.Logaritmic_Transform(radiograph, dimensions, pixel_size)
    htmps[i] = htmp_array

In [ ]:
slices = [-280, 245, 4] # initial, final, step
slices = [-280, -276, 4] # initial, final, step

reconstruction = RadLib.RadonReconstruction(roots, htmps, slices)

mu_water = 0.26748 # 40kev
mu_water = 0.18443 # 80kev
air_tolerance = -800
# HU_images = RadLib.CoefficientstoHU(reconstruction, slices, mu_water, air_tolerance)

## 8.4 Export to DICOM

In [ ]:
directory = '/Users/miguelcomett/geant4-v11.2.2_2/G4_MedX/OldRoots/CT_DICOM'

dimensions = [250, 75, 0, 150]
size_y = dimensions[1]

compressed = False

RadLib.export_to_dicom(HU_images, size_y, directory, compressed)